В качестве домашнего задания вам предлагается поработать над предсказанием погоды. Файл с данными вы найдете в соответствующей директории. Вам будет доступен датасет weather.csv, ПЕРВЫЕ 75% (shuffle = False) которого нужно взять для обучения, последние 25% - для тестирования.

Требуется построить 4 модели которые будут предсказывать целевую переменную <b>RainTomorrow</b> с помощью:

   1. логистической регрессии [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
   
   2. метода ближайших соседей [sklearn.neighbors](https://scikit-learn.org/stable/modules/neighbors.html)
 
   3. Байесовского классификатора [sklearn.naive_bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
   
   4. логистической регрессии реализованной самостоятельно

Затем следует сравнить результаты моделей (по качеству и времени выполнения) и сделать вывод о том, какая модель и с какими параметрами даёт лучшие результаты.

Не забывайте о том, что работа с признаками играет очень большую роль в построении хорошей модели.

Краткое описание данных:

    Date - Дата наблюдений
    Location - Название локации, в которой расположена метеорологическая станция
    MinTemp - Минимальная температура в градусах цельсия
    MaxTemp - Максимальная температура в градусах цельсия
    Rainfall - Количество осадков, зафиксированных за день в мм
    Evaporation - Так называемое "pan evaporation" класса А (мм) за 24 часа до 9 утра
    Sunshine - Число солнечных часов за день
    WindGustDir - направление самого сильного порыва ветра за последние 24 часа
    WindGustSpeed - скорость (км / ч) самого сильного порыва ветра за последние 24 часа
    WindDir9am - направление ветра в 9 утра

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import time
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

%matplotlib notebook

In [ ]:
X = pd.read_csv('weather.csv')

In [ ]:
y = X.RainTomorrow.replace({'No':-1, 'Yes': 1})

In [ ]:
del X['RainTomorrow']

### Реализация логистической регрессии
__Логистическая регрессия__

$$p(y|x) = a(x, \theta) = \sigma(\langle x, \theta \rangle) = \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}$$

In [ ]:
theta = np.array([1, 2, 3])

X =  np.array([[ 1,  1, 1],
               [-1, -2, 1],
               [-1, -2, 2],
               [-2, -2, -3]
              ])

y = np.array([1, 1, -1, -1])

In [ ]:
def probability(theta, X):
    # HERE MY CODE BEGINS
    result = 1.0 / (1 + np.exp( - np.dot(X, theta)))
    # END OF MY CODE
    return result
prob = probability(theta, X)

assert type(prob) == np.ndarray, 'Возвращается неверный тип'
assert prob.shape == (X.shape[0],), 'Неверный размер массива'
assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'

Функция предсказания метки класса, получает на вход вероятности принадлежности к классу 1 и выдает метки классов $y \in \{0, 1\}$

In [ ]:
def binary_class_prediction(theta, X, threshold =.5):
    prob =  probability(theta, X)
    # HERE MY CODE BEGINS
    result = (prob < threshold).astype(int)
    #result = 2* ((prob < threshold).astype(int)) - 1
    # END OF MY CODE
    return result

y_pred = binary_class_prediction(theta, X)


assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'
assert y_pred.shape == (X.shape[0],), 'Неверный размер массива'
assert min(y_pred) == 0, 'Функция считается неверно'
assert max(y_pred) == 1, 'Функция считается неверно'

__Функционал качества логистической регрессии__

Запишем правдободовие выборки для меток класса $y \in \{+1, -1\}$ 

$$Likelihood(a, X^\ell) = \prod_{i = 1}^{\ell} a(x_i,\theta)^{[y_i = +1]} (1 - a(x_i, \theta))^{[y_i = -1]} → \operatorname*{max}_{\theta}$$ 

Прологарифмируем правдоподобие выборки и перейдем к задаче минимизации:

$$Q(a, X^\ell) =     -\sum_{i = 1}^{\ell} 
        [y_i = +1] \log a(x_i, \theta)
        +
        [y_i = -1] \log (1 - a(x_i, \theta)) \to \operatorname*{min}_{\theta}$$ 
        
Подставим $a(x, \theta)$ в функцинал качества:

$$ Q(a, X^\ell) = -\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{\exp(-\langle \theta, x_i \rangle)}{1 + \exp(-\langle \theta, x_i \rangle)}
\right)
=\\
=
-\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{1}{1 + \exp(\langle \theta, x_i \rangle)}
\right)
=\\
=
\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) $$
    

Итоговый оптимизируемый функционал качества (logloss), записанный для меток классов $y \in \{+1, -1\}$ и усредненный по выборке

$$Q(a, X^\ell) = \frac{1}{\ell}\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) \to \operatorname*{min}_{\theta}$$

Реализуем его в функции logloss:

In [ ]:
def logloss(theta, X, y): 
    # YOUR CODE HERE
    return np.average(np.log(1 + np.exp(-y*(np.dot(X, theta)))))

In [ ]:
assert logloss(theta, X, y).round(3) == 0.861, 'Функция считается неверно'

__Алгоритм оптимизации функционала качества. Стохастический градиентный спуск__

<b>Вход: </b> Выборка $X^\ell$, темп обучения $h$

<b>Выход: </b> оптимальный вектор весов $\theta$

1.  Инициализировать веса $\theta$
2.  Инициализировать оценку функционала качества: $Q(a, X^\ell)$
3.  <b>Повторять</b>: 

    Выбрать случайным образом подвыборку объектов $X^{batch} =\{x_1, \dots,x_n \}$ из $X^{\ell}$
    
    Рассчитать градиент функционала качества: $\nabla Q(X^{batch}, \theta)$
    
    Обновить веса: $\theta := \theta - h\cdot \nabla Q(X^{batch}, \theta)$
       
    <b>Пока</b> значение $Q$ и/или веса $\theta$ не сойдутся   

Реализуем функцию рассчета градиента функционала качества

$$\frac{\partial Q(a, X^{batch}) }{\partial \theta_j}   = \frac{\partial \frac{1}{n}\sum_{i = 1}^{n}
    \log \left(
        1 + \exp(- y_i \langle \theta, x_i \rangle)
    \right)} {\partial \theta_j}  = \frac{1}{n}\sum_{i = 1}^{n}
     \frac {1}{
        1 + \exp(- y_i \langle \theta, x_i \rangle)} \cdot  \exp(- y_i \langle \theta, x_i \rangle) \cdot -y_i x_{ij}$$

Реализуйте рассчет градиента в матричном виде:

In [ ]:
def gradient(theta, X, y):
    # YOUR CODE HERE
    return np.average((-y * X.T/(1+np.exp(-y*(np.dot(X, theta)))) * np.exp(-y*(np.dot(X, theta)))).T, axis = 0)

assert gradient(theta, X, y).shape == theta.shape, 'Неверный размер массива'

Функция обучения уже реализована

In [ ]:
def fit(X, y, batch_size=10, h=0.05,  iters=100, plot=True):

    # получаем размерности матрицы
    size, dim = X.shape

    # случайная начальная инициализация
    theta = np.random.uniform(size=dim)
    errors = []
    
    theta_history = theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
    # plt 
    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')
        
        
    for _ in range(iters):  
        
        # берём случайный набор элементов
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]
        # считаем производные
        grad = gradient(theta, X_batch, y_batch)
        
        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'
        
        # Обновляем веса
        
        theta -= grad * h
        
        theta_history = np.vstack((theta_history, theta))
        
        # error
        loss = logloss(theta, X, y)
        errors.append(loss)
        
        if plot:
            ax1.clear()            
            ax1.scatter(range(dim), theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')
            
            
            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')
            
            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()   
            plt.show()

    return theta

In [ ]:
X, y = make_classification(n_samples=2000)

In [ ]:
optimal_theta = fit(X, y)
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
y_pred = binary_class_prediction(optimal_theta, X)


Таким образом, реализация логистической регрессии имеет вид:

In [ ]:
class log_regr:      
    def probability(self, X):
        result = 1/(1+np.exp(- np.dot(X, self.theta)))
        return result

    def binary_class_prediction(self, X, threshold =.5, anticlass = 0):
        prob =  self.probability(X)
        if anticlass == 0:
          result = (prob < threshold).astype(int)
        else:
          result = 2* ((prob < threshold).astype(int)) - 1
        return result

    def logloss(self, X, y): 
        return np.average(np.log(1 + np.exp(-y*(np.dot(X, self.theta)))))

    def gradient (self, X, y):
        return np.average((-y * X.T/(1+np.exp(-y*(np.dot(X, self.theta)))) * np.exp(-y*(np.dot(X, self.theta)))).T, axis = 0)
   # def exp_(self, i, X, y):
   #     return  np.exp(-y[i]*np.dot(X[i], self.theta))
  
  #  def gradient(self, X, y):
  #      summa = np.zeros_like(y)
  #      for i in range(len(y)):
  #        exp__ = self.exp_(i, X, y)
  #        summa += exp__/(1 + exp__)*(-y[i]*X[i])
  #      return summa / len(y) 

    def fit(self, X, y, batch_size=10, h=0.05,  iters=100, plot=False):
        # получаем размерности матрицы
        size, dim = X.shape
        # случайная начальная инициализация
        self.theta = np.random.uniform(size=dim)
        errors = []
        theta_history = self.theta
        colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
        # plt 
        if plot:
            fig = plt.figure(figsize=(15, 10))
            ax1 = fig.add_subplot(221)
            ax2 = fig.add_subplot(222)
            ax3 = fig.add_subplot(212)
            fig.suptitle('Gradient descent')
        
        
        for _ in range(iters):  
        
            # берём случайный набор элементов
            batch = np.random.choice(size, batch_size, replace=False)
            X_batch = X.iloc[batch]
            y_batch = y.iloc[batch]

            # считаем производные
            grad = self.gradient(X_batch, y_batch)
        
            assert type(grad) == np.ndarray, 'неверный тип'
            assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
            assert grad.shape[0] == len(self.theta), 'длина вектора должна быть равной количеству весов'
        
        
            # Обновляем веса
        
            self.theta -= grad * h
        
            theta_history = np.vstack((theta_history, self.theta))
        
            # error
            loss = self.logloss(X, y)
            errors.append(loss)
        
            if plot:
                ax1.clear()            
                ax1.scatter(range(dim), self.theta, label='Gradient solution')
                ax1.legend(loc="upper left")
                ax1.set_title('theta')
                ax1.set_ylabel(r'$\bar \beta$')
                ax1.set_xlabel('weight ID')
            
            
                ax2.plot(range(_+1), errors, 'g-')
                ax2.set_title('logloss')
                ax2.set_xlabel('itarations')
            
                ax3.plot(theta_history)
                ax3.set_title('update theta')
                ax3.set_ylabel('value')
                ax3.set_xlabel('itarations')
                time.sleep(0.05)
                fig.canvas.draw()   
            
        return

Проверим результат для логистической регрессии, реализованной выше:

In [ ]:
X = pd.read_csv('weather.csv')
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})
del X['RainTomorrow']
del X['Unnamed: 0']

X = X.replace({'No':0, 'Yes': 1})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

num_data = X_train.select_dtypes([np.number])

num_data_mean = num_data.mean()
num_features = num_data.columns

X_train = X_train.fillna(num_data_mean)
X_test = X_test.fillna(num_data_mean)


In [ ]:
%%time
model = log_regr()
model.fit(X_train[num_features], y_train)

y_pred = model.binary_class_prediction(X_test[num_features], anticlass=0)
y_pred_train = model.binary_class_prediction(X_train[num_features], anticlass=0)


CPU times: user 1.99 s, sys: 1.44 s, total: 3.43 s
Wall time: 1.79 s


In [ ]:
print("Test MAE = ", mean_absolute_error(y_test, y_pred))
print("Train MAE = ", mean_absolute_error(y_train, y_pred_train))
print("Accuracy = ", accuracy_score(y_pred, y_test))

Test MAE =  0.22349433176741962
Train MAE =  0.2244101871647725
Accuracy =  0.7765056682325804


# Логистическая регрессия (готовая)

Теперь используем модель из модуля:


In [ ]:
X = pd.read_csv('weather.csv')
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})
del X['RainTomorrow']
del X['Unnamed: 0']

X = X.replace({'No':0, 'Yes': 1})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

num_data = X_train.select_dtypes([np.number])

num_data_mean = num_data.mean()
num_features = num_data.columns

X_train = X_train.fillna(num_data_mean)
X_test = X_test.fillna(num_data_mean)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
%time
LogRegr = LogisticRegression()
LogRegr.fit(X_train[num_features], y_train)

y_pred = LogRegr.predict(X_test[num_features])
y_train_pred = LogRegr.predict(X_train[num_features])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 9.54 µs


In [ ]:
print(classification_report(y_test, LogRegr.predict(X_test[num_features])))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90     27604
           1       0.71      0.46      0.56      7945

    accuracy                           0.84     35549
   macro avg       0.79      0.71      0.73     35549
weighted avg       0.83      0.84      0.83     35549



In [ ]:
print("Test MAE =", mean_absolute_error(y_test, y_pred))
print("Train MAE =", mean_absolute_error(y_train, y_train_pred))
print("Accuracy =", accuracy_score(y_pred, y_test))

Test MAE = 0.16129848940898478
Train MAE = 0.16053411349911856
Accuracy = 0.8387015105910152


# Метод ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
X = pd.read_csv('weather.csv')
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})
del X['RainTomorrow']
del X['Unnamed: 0']

X = X.replace({'No':0, 'Yes': 1})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

num_data = X_train.select_dtypes([np.number])

num_data_mean = num_data.mean()
num_features = num_data.columns

X_train = X_train.fillna(num_data_mean)
X_test = X_test.fillna(num_data_mean)

In [ ]:
%%time
n = 10
model = KNeighborsClassifier(n_neighbors=n)
model.fit(X_train[num_features], y_train)

knn_pred = model.predict(X_test[num_features])
knn_pred_train = model.predict(X_train[num_features])

CPU times: user 5min 21s, sys: 17.9 s, total: 5min 39s
Wall time: 5min 4s


In [ ]:
print(classification_report(y_test, model.predict(X_test[num_features])))

              precision    recall  f1-score   support

           0       0.86      0.96      0.90     27604
           1       0.74      0.44      0.56      7945

    accuracy                           0.84     35549
   macro avg       0.80      0.70      0.73     35549
weighted avg       0.83      0.84      0.83     35549



In [ ]:
print("Test MAE =", mean_absolute_error(y_test, knn_pred))
print("Train MAE =", mean_absolute_error(y_train, knn_pred_train))
print("Accuracy =", accuracy_score(y_test, knn_pred))

Test MAE = 0.15868238206419308
Train MAE = 0.13933273320580625
Accuracy = 0.841317617935807


# Байесовский классификатор

In [ ]:
from sklearn.naive_bayes import GaussianNB


In [ ]:
X = pd.read_csv('weather.csv')
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})
del X['RainTomorrow']
del X['Unnamed: 0']

X = X.replace({'No':0, 'Yes': 1})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

num_data = X_train.select_dtypes([np.number])

num_data_mean = num_data.mean()
num_features = num_data.columns

X_train = X_train.fillna(num_data_mean)
X_test = X_test.fillna(num_data_mean)

In [ ]:
%%time
model = GaussianNB()
model.fit(X_train[numeric_features], y_train)
y_pred = model.predict(X_test[numeric_features])
y_train_pred = model.predict(X_train[numeric_features])

CPU times: user 125 ms, sys: 11.5 ms, total: 136 ms
Wall time: 160 ms


In [ ]:
print("Test MAE =", mean_absolute_error(y_test, y_pred))
print("Train MAE =", mean_absolute_error(y_train, y_train_pred))
print("Accuracy =", accuracy_score(y_test, y_pred))

Test MAE = 0.1967
Train MAE = 0.1951
Accuracy = 0.8033


# Выводы

1. Логистическая регрессия, реализованная вручную:

    time: 3.43 s 

    Test MAE ~ Train MAE ~ 0.22

    Accuracy: 0.78
2. Готовая логистическая регрессия:

    time: 3 µs

    Test MAE ~ Train MAE ~ 0.16

    Accuracy: 0.84
3. Метод ближайших соседей:

    time: 5min 39s

    Test MAE ~ 0.16 
    Train MAE ~ 0.14

    Accuracy: 0.84
4. Байесовский классификатор:

    time: 136 ms
    
    Test MAE ~ Train MAE ~ 0.20

    Accuracy = 0.80

Метод k ближайших соседей наиболее медленный из представленных здесь. Наибольшая скорость работы у готовой логистической регрессии. 